In [186]:
import pandas as pd
import numpy as np
import random

from lxml import objectify
import json
from pandas.io.json import json_normalize
from pandas.io.pytables import HDFStore
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings("ignore")

In [3]:
#Datasets
mtcars = pd.read_csv('datasets/mtcars.csv')
salaries = pd.read_csv('datasets/Salaries.csv')
beml = pd.read_csv('datasets/BEML.csv')
glaxo = pd.read_csv('datasets/GLAXO.csv')
titanic = pd.read_csv('datasets/train.csv')

In [7]:
#-------------------I/O API Tools----------------------------------
#Readers
#read_csv, read_excel, read_hdf, read_sql, read_json, read_html, read_stata, read_clipboard, read_pickle, read_msgpack, read_gbq

#Writers
#to_csv, to_excel, to_hdf, to_sql, to_json, to_html, to_stata, to_clipboard, to_pickle, to_msgpack(experimental), to_gbq(experimental)

In [8]:
#-------------------CSV and Textual Files-----------------
#f the values in a row are separated by a comma, you have the CSV (comma-separated values) format, 
# which is perhaps the best-known and most popular format.
#Other forms with tabular data separated by spaces or tabs are typically contained in text files of various 
# types (generally with the extension .txt).

In [9]:
#-----------------Reading Data in CSV or Text Files---------------------

In [17]:
df = pd.read_csv("bookdatasets/myCSV_01.csv")
df
#But since CSV files are considered text files, you can also use the read_table() function, but specifying the delimiter.
df = pd.read_table("bookdatasets/myCSV_01.csv",sep=",")
df

,white,red,blue,green,animal
0,1,5,2,3,cat
1,2,7,8,5,dog
2,3,3,6,7,horse
3,2,2,8,3,duck
4,4,4,2,1,mouse


In [25]:
#When headers is missing
#In this case, then you could make sure that it is precisely pandas to assign default names to the columns 
# by using the header option set to None.
df = pd.read_csv("bookdatasets/mycsv_02.csv",header=None)
df

#In addition, there is also the possibility to specify the names directly assigning a list of labels to the names option.
df = pd.read_csv("bookdatasets/mycsv_02.csv",names=["white","red","blue","green","animal"])
df

,white,red,blue,green,animal
0,1,5,2,3,cat
1,2,7,8,5,dog
2,3,3,6,7,horse
3,2,2,8,3,duck
4,4,4,2,1,mouse


In [31]:
# In more complex cases, in which you want to create a DataFrame with a hierarchical structure by 
# reading a CSV file, you can extend the functionality of the read_csv() function by adding the index_col
# option, assigning all the columns to be converted into indexes to it.

df = pd.read_csv("bookdatasets/myCSV_03.csv",index_col=["color","status"])
df

item1  item2  item3
color status                     
black up          3      4      6
      down        2      6      7
white up          5      5      5
      down        3      3      2
      left        1      2      1
red   up          2      2      2
      down        1      1      4

In [36]:
#-----Using RegExp for Parsing TXT Files------
# suppose that your file, such as a TXT file, has values separated by spaces or tabs in an unpredictable order.
# In this case, you have to use the regexp because only with it you will take into account as a separator both cases.
# You can do that using the wildcard /s*. /s stands for space or tab character (if you wanted to indicate only the tab,
# you would have used /t), while the pound indicates that these characters may be multiple (see Table 5-1 for other 
# wildcards most commonly used). That is, the values may be separated by more spaces or more tabs.
df = pd.read_table("bookdatasets/myCSV_04.csv",sep="\s")
df

,white,red,blue,green
0,1,5,2,3
1,2,7,8,5
2,3,3,6,7


In [47]:
df = pd.read_table("bookdatasets/myCSV_05.txt",sep="\D{3}",header=None)
df

,0,1,2
0,0,123,122
1,1,124,321
2,2,125,333


In [49]:
#Another fairly common event is to exclude lines from parsing.
#Pay attention when you are using this option. If you want to exclude the first five lines, then you have to 
# write skiprows = 5, but if we want to rule out the fifth line you have to write skiprows = [5].
df = pd.read_table("bookdatasets/myTXT_01.txt",sep=",",skiprows=[0,1,3,6])
df

,white,red,blue,green,animal
0,1,5,2,3,cat
1,2,7,8,5,dog
2,3,3,6,7,horse
3,2,2,8,3,duck
4,4,4,2,1,mouse


In [51]:
#------------------Reading TXT Files into Parts or Partially-----------------
#When large files are processed, or when you’re only interested in portions of these files, you often need to 
# read the file into portions (chunks).

#if we want to read only a portion of the file, you can explicitly specify the number of 
# lines on which to parse. Thanks to the nrows and skiprows options, you can select the starting line 
# n (n = SkipRows) and the lines to be read after it (nrows = i).
df = pd.read_table("bookdatasets/myTXT_01.txt",sep=",",skiprows=[0,1,3,6],nrows=3)
df

,white,red,blue,green,animal
0,1,5,2,3,cat
1,2,7,8,5,dog
2,3,3,6,7,horse


In [68]:
# Another interesting and fairly common operation is to split into portions that part of the text on which 
# we want to parse. Then for each portion a specific operation may be carried out, in order to obtain an 
# iteration, portion by portion.
ser = pd.Series([])
i = 0
pieces = pd.read_table("bookdatasets/myCSV_01.csv",sep=",",chunksize=3)
for piece in pieces:
    ser.at[i] = piece["red"].sum()
    i = i + 1
    
ser

0    15
1     6
dtype: int64

In [75]:
ser = pd.Series()
pieces = pd.read_csv("datasets/Salaries.csv",chunksize=6)
i = 0
for piece in pieces:
    ser.at[i]=piece["salary"].max()
    i = i + 1
     
ser


0     186960
1     134778
2     162200
3     150500
4     155865
5     155750
6     146856
7     151768
8     127512
9     144651
10    117555
11    116450
12    161101
dtype: int64

In [83]:
#-----------------Writing Data in CSV------------------------
df = pd.DataFrame(np.arange(16).reshape(4,4),columns=["ball","pen","pencil","paper"])

# As you can see from the previous example, when you make the writing of a data frame to a file, by 
# default both indexes and columns are marked on the file. This default behavior can be changed by placing 
# the two options index and header set to False .

df.to_csv("bookdatasets/genCSV_01.csv",index=False)

In [84]:
df = pd.read_csv("bookdatasets/genCSV_01.csv")
df

,ball,pen,pencil,paper
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15


In [88]:
df = pd.DataFrame(np.round(np.random.random((4,4))*10,0),dtype="int32")
df.to_csv("bookdatasets/genCSV_02.csv",index=False,header=False)

In [91]:
df = pd.read_csv("bookdatasets/genCSV_02.csv",header=None)
df

,0,1,2,3
0,3,3,5,9
1,7,5,3,3
2,7,2,8,5
3,8,3,1,9


In [126]:
# One thing to take into account when making the writing of files is that NaN values present in a data 
# structure are shown as empty fields in the file
ls = np.log(np.array([random.randint(-10,10)for i in range(25)])).reshape(5,5)
df = pd.DataFrame(ls,index=["blue","green","red","white","yellow"],columns=["ball","mug","paper","pen","pencil"])

# But you can replace this empty field with a value to your liking using the na_rep option in the to_csv()
# function. Common values may be NULL, 0, or the same NaN.

df.to_csv("bookdatasets/genCSV_03.csv",na_rep=0)

In [127]:
df = pd.read_csv("bookdatasets/genCSV_03.csv",index_col=[0])
# df.reindex(df["Unnamed:0"])
df


,ball,mug,paper,pen,pencil
blue,1.791759,1.386294,0.0,1.386294,0.000000
green,0.000000,0.000000,0.0,0.000000,0.693147
red,2.079442,0.000000,0.0,2.197225,0.000000
white,1.609438,-inf,-inf,0.693147,1.098612
yellow,0.000000,1.945910,-inf,1.386294,0.000000


In [ ]:
#----------------Reading and Writing HTML Files------------------------
# This activity is so widespread that it is currently identified as Web Scraping. This process is becoming 
# a fundamental part of the set of processes that will be integrated in the first part of the data analysis: data 
# mining and data preparation.

In [128]:
#---------------Writing Data in HTML-----------------------

In [133]:
df = pd.DataFrame(np.arange(4).reshape(2,2))
print(df.to_html(index=False,header=False))

<table border="1" class="dataframe">
  <tbody>
    <tr>
      <td>0</td>
      <td>1</td>
    </tr>
    <tr>
      <td>2</td>
      <td>3</td>
    </tr>
  </tbody>
</table>


In [135]:
df = pd.DataFrame( np.random.random((4,4)),
                     index = ['white','black','red','blue'],
                     columns = ['up','down','right','left'])
print(df.to_html())

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>up</th>
      <th>down</th>
      <th>right</th>
      <th>left</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>white</th>
      <td>0.849857</td>
      <td>0.750780</td>
      <td>0.429793</td>
      <td>0.782152</td>
    </tr>
    <tr>
      <th>black</th>
      <td>0.421484</td>
      <td>0.496586</td>
      <td>0.102234</td>
      <td>0.157546</td>
    </tr>
    <tr>
      <th>red</th>
      <td>0.653572</td>
      <td>0.906469</td>
      <td>0.515452</td>
      <td>0.091790</td>
    </tr>
    <tr>
      <th>blue</th>
      <td>0.042973</td>
      <td>0.502853</td>
      <td>0.121878</td>
      <td>0.310400</td>
    </tr>
  </tbody>
</table>


In [136]:
#----------------Reading Data from an HTML File--------------

In [140]:
#More precisely, the read_html() function returns a list of DataFrame even if there is only one table.
web_df = pd.read_html("bookdatasets/test.html",index_col=[0])
web_df[0]

,up,down,right,left
white,0.849857,0.750780,0.429793,0.782152
black,0.421484,0.496586,0.102234,0.157546
red,0.653572,0.906469,0.515452,0.091790
blue,0.042973,0.502853,0.121878,0.310400


In [143]:
# However, the mode most commonly used regarding the read_html() function is that of a direct parsing 
# of an URL on the Web. In this way the web pages in the network are directly parsed with the extraction of the 
# tables within them.
web_df = pd.read_html("http://www.meccanismocomplesso.org/en/meccanismo-complesso-sito-2/classifica-punteggio/",index_col=[0])
web_df[0]

,Member,Points,Levels
1,BrunoOrsini,2075,NaN
2,Berserker,700,NaN
3,albertosallu,275,NaN
4,Jon,180,NaN
5,Mr.Y,180,NaN
...,...,...,...
111,Gigi Bertana,5,NaN
112,p.barut,5,NaN
113,Indri4Africa,5,NaN
114,ghirograf,5,NaN


In [153]:
#--------------------Reading Data from XML----------------------
# Now you can do the parser of the XML file with just the parse() function.
xml = objectify.parse("bookdatasets/book.xml")
xml
#You got an object tree, which is an internal data structure of the module lxml.
root = xml.getroot()
root
# Now that the root of the structure has been defined, you can access the various nodes of the tree, each 
# corresponding to the tag contained within the original XML file.
root.Book.Author
root.Book.PublishDate
#In this way you access nodes individually, but you can access various elements at the same time using getchildren().
root.getchildren()
#With the tag attribute you get the name of the tag corresponding to the child node.
[child.tag for child in root.Book.getchildren()]
#while with the text attribute you get the value contained between the corresponding tags.
[child.text for child in root.Book.getchildren()]

#Define the function, which has the task of analyzing the entire 
# contents of a eTree to fill a DataFrame line by line.

['272103_1_EnRoss, Mark', 'XML Cookbook', 'Computer', '23.56', '2014-22-01']

In [162]:
#------------------Reading and Writing Data on Microsoft Excel Files----------
sheet1_df = pd.read_excel("bookdatasets/data.xlsx",index_col=[0])
sheet1_df
# As you can see, by default, the returned DataFrame is composed of the data tabulated in the first 
# spreadsheets. If, however, you’d need to load the data in the second spreadsheet, and then specify the name 
# of the sheet or the number of the sheet (index) just as the second argument.
sheet2_df = pd.read_excel("bookdatasets/data.xlsx","Sheet2",index_col=[0])
sheet2_df

,yellow,purple,blue,orange
A,11,16,44,22
B,20,22,23,44
C,30,31,47,32


In [163]:
#To write to excel sheet
df = pd.DataFrame(np.random.random((4,4)),
             index = ['exp1','exp2','exp3','exp4'],
             columns = ['Jan2015','Fab2015','Mar2015','Apr2005'])
df.to_excel("bookdatasets/data2.xlsx")

In [165]:
#-------------------JSON (JavaScript Object Notation) Data--------------------------

In [166]:
###############   NOTE ###################
# In my opinion, a useful online application for checking the JSON format is JSONViewer, available at 
# http://jsonviewer.stack.hu/. This web application, once you entered or copied data in JSON format, 
# allows you to see if the format you entered is invalid. Moreover it displays the tree structure so that you can 
# better understand its structure

In [167]:
#writing JSON file
df = pd.DataFrame(np.arange(16).reshape(4,4),
                  index=['white','black','red','blue'],
                  columns=['up','down','right','left'])
df.to_json("bookdatasets/frame.json")

In [168]:
#REading JSON To DataFrame
df = pd.read_json("bookdatasets/frame.json")
df

,up,down,right,left
white,0,1,2,3
black,4,5,6,7
red,8,9,10,11
blue,12,13,14,15


In [170]:
#Generally, however, the JSON files do not have a tabular structure. Thus, you will need to somehow convert the structure
# dict file in tabular form. You can refer this process as normalization.

#The library pandas provides a function, called json_normalize(), that is able to convert a dict or a list in 
# a table. First you have to import the function

In [176]:
with open("bookDatasets/books.json","r") as file:
    text = file.read()
    text = json.loads(text)
json_normalize(text,"books",["writer","nationality"])

,title,price,writer,nationality
0,XML Cookbook,23.56,Mark Ross,USA
1,Python Fundamentals,50.70,Mark Ross,USA
2,The NumPy library,12.30,Mark Ross,USA
3,Java Enterprise,28.60,Barbara Bracket,UK
4,HTML5,31.35,Barbara Bracket,UK
5,Python for Dummies,28.00,Barbara Bracket,UK


In [182]:
#--------------------The Format HDF5(Hierarchical Data Format)---------------------------
df = pd.DataFrame(np.arange(16).reshape(4,4),
                  index=['white','black','red','blue'],
                  columns=['up','down','right','left'])
df2 = pd.DataFrame(np.random.random(16).reshape(4,4),
                  index=['white','black','red','blue'],
                  columns=['up','down','right','left'])
store = HDFStore("bookDatasets/mydata.h5")
store["obj1"] = df
store["obj2"] = df2
#So with this type of format, you can store multiple data structures within a single file, represented by the store variable.

In [185]:
store["obj2"]

,up,down,right,left
white,0.211953,0.843458,0.327681,0.255733
black,0.576480,0.381122,0.936325,0.979081
red,0.355879,0.230515,0.158284,0.821641
blue,0.676549,0.949859,0.850193,0.796422


In [187]:
#--------- Interacting with Databases-------------------------
# The pandas.io.sql module provides a unified interface independent of the DB, called sqlalchemy. This 
# interface simplifies the connection mode, since regardless of the DB, the commands will always be the same. 
# For making a connection you use the create_engine() function. With this feature you can configure all the 
# properties necessary to use the driver, as a user, password, port, and database instance.

In [188]:
#Now it’s time to implement the connection to the SQLite3 database.
engine = create_engine('sqlite:///foo.db')

In [194]:
#-----------------------Loading and Writing Data with SQLite3-------------------
df = pd.DataFrame( np.arange(20).reshape(4,5),
                     columns=['white','red','blue','black','green'])
#Convert the DataFrame in a table within the database.
df.to_sql("table_name1",engine,index=False)

4

In [195]:
#to make a reading of the database, you have to use the read_sql() function with the name of the table and the engine.
pd.read_sql("table_name1",engine)

,white,red,blue,black,green
0,0,1,2,3,4
1,5,6,7,8,9
2,10,11,12,13,14
3,15,16,17,18,19


In [ ]:
#Even here there is a read_sql_table()function that reads directly on the database and returns a DataFrame.
pd.read_sql_table('dataframe',engine)

# But when you want to make a reading of data in a database, the conversion of a whole and single table 
# into a DataFrame is not the most useful operation. In fact, those who work with relational databases prefer to 
# use the SQL language to choose what data and in what form to export by inserting an SQL query.
# The text of an SQL query can be integrated in the read_sql_query() function.
pd.read_sql_query('SELECT index,apr,may FROM DATAFRAME WHERE apr > 0.5',engine)